In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
import json

In [14]:
with open("prospect_to_docs.json", "r") as f:
    prospects = json.load(f)

In [15]:
with open("curr_player_to_docs.json", "r") as f:
    curr_players = json.load(f)

In [7]:
def get_tfidf_matrix(player_dict):
    text_list = []
    playername_list = []
    for player, text in player_dict.items():
        text_list.append(text)
        playername_list.append(player)

    tfidf_vectorizer = TfidfVectorizer(input=text_list, lowercase=True, stop_words="english", max_df=0.8)
    tfidf_text = tfidf_vectorizer.fit_transform(text_list)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    return tfidf_text, tfidf_feature_names

In [16]:
tfidf_prospects_matrix, tfidf_prospects_feature_names = get_tfidf_matrix(prospects)
print(tfidf_prospects_matrix.shape)

tfidf_curr_players_matrix, tfidf_curr_players_feature_names = get_tfidf_matrix(curr_players)
print(tfidf_curr_players_matrix.shape)


(100, 5584)
(719, 19866)


In [12]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

def topic_model(tfidf_matrix, tfidf_feature_names):
    lda_model = LatentDirichletAllocation(n_topics=10).fit(tfidf_matrix)
    display_topics(lda_model, tfidf_feature_names, no_top_words=15)
    
#     lda_text = lda_model.transform(tfidf_matrix)
    

In [17]:
topic_model(tfidf_curr_players_matrix, tfidf_curr_players_feature_names)

C:\Users\Eric\Anaconda2\envs\cs4300\lib\site-packages\sklearn\decomposition\online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic 0:
varnado patterson horford bradley felton ager ennis amir majok wright video vs trying analysis uconn
Topic 1:
johnson video interview workout booker post vs motiejunas carney bairstow gallon guard allen payne paul
Topic 2:
thornton wright burks mack love crittenton hanlan guard strawberry thompkins gist macklin connaughton radicevic singletary
Topic 3:
thabeet lee summers martin workout preview ooze robinson 0wingspan inspiration masking hammer semblances sandwiched unlucky
Topic 4:
hobson lowry karasev ulis cousins oden mayo ilyasova jenkins report video scouting grant tavares breakdown
Topic 5:
hamilton almond hilliard barac papanikolaou daye barton video green price liggins lalanne scouting randle daniels
Topic 6:
withey sharpe kanter rivers bryant griffin hanga ewing millsap lawson norel caldwell zeller noel johnson
Topic 7:
foye yi wall douby dozier vaden lucas brown harrison nogueira uconn video lot going ll
Topic 8:
interview noah rush workout richard video rozier walla